In [143]:
import torch 
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [144]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [145]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [146]:
df.drop(columns=['id','Unnamed: 32'],inplace=True, axis=1)

In [147]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [148]:
df.duplicated().sum()

np.int64(0)

In [149]:
import numpy as np
X=df.iloc[: ,1:]
Y=df.iloc[:,0]


In [150]:
X_train,X_test,y_train,y_test=train_test_split(X,Y, test_size=0.3)

In [151]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

label=LabelEncoder()
y_train=label.fit_transform(y_train)
y_test=label.transform(y_test)

In [152]:
X_train_tensor=torch.tensor(X_train)
X_test_tensor=torch.tensor(X_test)
y_train_tensor=torch.tensor(y_train)
y_test_tensor=torch.tensor(y_test)
X_train_tensor = X_train_tensor.to(torch.float32)
y_train_tensor = y_train_tensor.to(torch.float32)
X_test_tensor = X_test_tensor.to(torch.float32)
y_test_tensor = y_test_tensor.to(torch.float32)


In [153]:
from torch.utils.data import Dataset,DataLoader

class CustomDataset(Dataset):

    def __init__(self,features,labels):
        self.features=features
        self.labels=labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,idx):
        return self.features[idx],self.labels[idx]

In [154]:
train_dataset=CustomDataset(X_train_tensor,y_train_tensor)
test_dataset=CustomDataset(X_test_tensor,y_test_tensor)

In [155]:
train_dataset[10]

(tensor([-0.5465, -1.3707, -0.5279, -0.5641,  0.7430, -0.1793, -0.7572, -0.5307,
          0.2215,  0.2670, -0.7158, -0.7411, -0.4376, -0.5323, -0.1142, -0.0555,
         -0.6930, -0.6012,  0.6378,  0.1277, -0.5953, -1.2702, -0.4665, -0.5808,
          0.4652,  0.0314, -0.8210, -0.4974,  0.8761,  0.4630]),
 tensor(0.))

In [156]:
train_loader =DataLoader(train_dataset, batch_size=32,shuffle=True)
test_loader =DataLoader(test_dataset, batch_size=32,shuffle=True)

In [157]:
class MyModel(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.network=nn.Sequential(
            nn.Linear(num_features,15),
            nn.ReLU(),
            nn.Linear(15,5),
            nn.ReLU(),
            nn.Linear(5,3),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
    
    def forward(self,features):
        out=self.network(features)
        return out
    
    

In [158]:
learning_rate=0.1
epochs=50

In [159]:
loss_func=nn.BCELoss()

In [160]:
model=MyModel(X_train_tensor.shape[1])
#optimizer

optim=torch.optim.SGD(model.parameters(),lr=learning_rate)

for epochs in range(epochs):

    for batch_features, batch_labels in train_loader:

        y_pred=model(batch_features)

        loss=loss_func(y_pred,batch_labels.view(-1,1))

        optim.zero_grad()

        loss.backward()

        optim.step() #updating parameters

        print(f'Epoch:{epochs+1} , loss:{loss.item()}')

Epoch:1 , loss:0.7512751221656799
Epoch:1 , loss:0.694809079170227
Epoch:1 , loss:0.7314677834510803
Epoch:1 , loss:0.7504082918167114
Epoch:1 , loss:0.7677391767501831
Epoch:1 , loss:0.7392725944519043
Epoch:1 , loss:0.7184621691703796
Epoch:1 , loss:0.7479846477508545
Epoch:1 , loss:0.7224974632263184
Epoch:1 , loss:0.7071040868759155
Epoch:1 , loss:0.7281863689422607
Epoch:1 , loss:0.7323960065841675
Epoch:1 , loss:0.7156693339347839
Epoch:2 , loss:0.7014817595481873
Epoch:2 , loss:0.700435996055603
Epoch:2 , loss:0.6979866027832031
Epoch:2 , loss:0.6820181608200073
Epoch:2 , loss:0.6848375797271729
Epoch:2 , loss:0.6985023021697998
Epoch:2 , loss:0.6805312037467957
Epoch:2 , loss:0.6831246614456177
Epoch:2 , loss:0.6798282265663147
Epoch:2 , loss:0.6736394762992859
Epoch:2 , loss:0.673721194267273
Epoch:2 , loss:0.6713276505470276
Epoch:2 , loss:0.6668168306350708
Epoch:3 , loss:0.6525135040283203
Epoch:3 , loss:0.6601929664611816
Epoch:3 , loss:0.6418851613998413
Epoch:3 , loss:0.

In [161]:
print(batch_features.shape)


torch.Size([14, 30])


In [162]:
model.eval()
accuracy_list=[]

with torch.no_grad():
    for batch_features,batch_labels in test_loader:

        y_pred=model.forward(batch_features)
        y_pred=(y_pred>0.5).float()

        batch_accuracy=(y_pred.view(-1)==batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

overall_accuracy=sum(accuracy_list)/len(accuracy_list)
print(f'Accuracy:{overall_accuracy:.4f}')

Accuracy:0.9740
